http://www.pytorchtutorial.com/10-minute-pytorch-3/

In [1]:
import torch
from torch import nn, optim

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

In [2]:
batch_size = 32
learning_rate = 1e-2
num_epoches = 50

In [3]:
# 下载训练集 MNIST 手写数字训练集
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
# 定义简单的前馈神经网络
class Neuralnetwork(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(Neuralnetwork, self).__init__()
        self.layer1 = nn.Linear(in_dim, n_hidden_1)
        self.layer2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.layer3 = nn.Linear(n_hidden_2, out_dim)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x


model = Neuralnetwork(28 * 28, 300, 100, 10)
if torch.cuda.is_available():
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
for epoch in range(num_epoches):
    print('epoch {}'.format(epoch + 1))
    print('*' * 10)
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        img = img.view(img.size(0), -1)
        if torch.cuda.is_available():
            with torch.no_grad():
                img = img.cuda()            
                label = label.cuda()
        else:
            with torch.no_grad():
                img = img            
                label = label
        # 向前传播
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss.item() * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        running_acc += num_correct.item()
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                epoch + 1, num_epoches, running_loss / (batch_size * i),
                running_acc / (batch_size * i)))
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, running_loss / (len(train_dataset)), running_acc / (len(
            train_dataset))))
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for data in test_loader:
        img, label = data
        img = img.view(img.size(0), -1)
        if torch.cuda.is_available():
            with torch.no_grad():
                img = img.cuda()            
                label = label.cuda()
        else:
            with torch.no_grad():
                img = img            
                label = label
        out = model(img)
        loss = criterion(out, label)
        eval_loss += loss.item() * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        eval_acc += num_correct.item()
    print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
        test_dataset)), eval_acc / (len(test_dataset))))
    print()


epoch 1
**********
[1/50] Loss: 0.237788, Acc: 0.933125
[1/50] Loss: 0.245343, Acc: 0.932344
[1/50] Loss: 0.240234, Acc: 0.933438
[1/50] Loss: 0.242141, Acc: 0.932682
[1/50] Loss: 0.244409, Acc: 0.932396
[1/50] Loss: 0.244044, Acc: 0.932413
Finish 1 epoch, Loss: 0.244486, Acc: 0.932533
Test Loss: 0.274259, Acc: 0.924700

epoch 2
**********
[2/50] Loss: 0.237184, Acc: 0.935104
[2/50] Loss: 0.240332, Acc: 0.932969
[2/50] Loss: 0.241020, Acc: 0.933924
[2/50] Loss: 0.240444, Acc: 0.933073
[2/50] Loss: 0.241933, Acc: 0.933125
[2/50] Loss: 0.244858, Acc: 0.931927
Finish 2 epoch, Loss: 0.244147, Acc: 0.932283
Test Loss: 0.272003, Acc: 0.925600

epoch 3
**********
[3/50] Loss: 0.234901, Acc: 0.933438
[3/50] Loss: 0.243816, Acc: 0.932187
[3/50] Loss: 0.245799, Acc: 0.932743
[3/50] Loss: 0.243145, Acc: 0.933151
[3/50] Loss: 0.244990, Acc: 0.932521
[3/50] Loss: 0.245071, Acc: 0.932552
Finish 3 epoch, Loss: 0.243823, Acc: 0.932867
Test Loss: 0.271601, Acc: 0.925500

epoch 4
**********
[4/50] Loss:

[26/50] Loss: 0.248396, Acc: 0.930937
[26/50] Loss: 0.245133, Acc: 0.933385
[26/50] Loss: 0.244195, Acc: 0.933958
[26/50] Loss: 0.237315, Acc: 0.934792
[26/50] Loss: 0.238330, Acc: 0.934083
[26/50] Loss: 0.238493, Acc: 0.934080
Finish 26 epoch, Loss: 0.239564, Acc: 0.933783
Test Loss: 0.269310, Acc: 0.926200

epoch 27
**********
[27/50] Loss: 0.225791, Acc: 0.937708
[27/50] Loss: 0.230410, Acc: 0.937708
[27/50] Loss: 0.232581, Acc: 0.935764
[27/50] Loss: 0.233523, Acc: 0.935182
[27/50] Loss: 0.235778, Acc: 0.934667
[27/50] Loss: 0.237747, Acc: 0.934288
Finish 27 epoch, Loss: 0.239013, Acc: 0.933950
Test Loss: 0.273720, Acc: 0.925300

epoch 28
**********
[28/50] Loss: 0.249180, Acc: 0.930833
[28/50] Loss: 0.237613, Acc: 0.932656
[28/50] Loss: 0.237600, Acc: 0.933299
[28/50] Loss: 0.237980, Acc: 0.933177
[28/50] Loss: 0.237188, Acc: 0.933187
[28/50] Loss: 0.238918, Acc: 0.933299
Finish 28 epoch, Loss: 0.238863, Acc: 0.933300
Test Loss: 0.273398, Acc: 0.925500

epoch 29
**********
[29/50]

Test Loss: 0.277231, Acc: 0.924700

